<a href="https://colab.research.google.com/github/apoorva2810/CS6910/blob/main/Assignment-2/partB/partB_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Before executing the code, allocate GPU in Colab from: Edit -> Notebook Settings -> Hardware accelerator -> GPU

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense,BatchNormalization
from keras.layers import Flatten
from keras.layers import Dropout
import random
import os
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm
from PIL import Image
import pickle
from keras.models import Model
from keras.layers import Input
from tensorflow.keras import layers, models, Model, optimizers
import tensorflow.keras as K

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
import tensorflow.keras as K



#Model
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import ResNet50
from keras.applications import InceptionResNetV2
from keras.applications import Xception



In [ ]:
#To access content from google drive
drive.mount('/content/gdrive')

In [ ]:
#Todo- first rename the file in drive
#Use this link: https://drive.google.com/drive/folders/1yu5DuYlM_cvV5qwIKKnl3sHRG_G41aLB?usp=sharing
#To make copy of pickled data and save it folder name 'Raw_data350'

with open('/content/gdrive/My Drive/Raw_data350/xtrain','rb') as xT:
  xTrain = pickle.load(xT)

with open('/content/gdrive/My Drive/Raw_data350/ytrain','rb') as yT:
  yTrain = pickle.load(yT)

with open('/content/gdrive/My Drive/Raw_data350/xtest','rb') as xV:
  xTest = pickle.load(xV)

with open('/content/gdrive/My Drive/Raw_data350/ytest','rb') as yV:
  yTest = pickle.load(yV)